<h1>Image Segmentation using DGPs</h1>

<p># Motivation: <br>
1) To solve the image segmentation task along with the uncertainity incorporation in this process using bayesian statistics<br>
2) Non-parametric approach to solve the problem.(millions of parameters are not desired for a flexible dataset training)<br></p>
<p># Image segmentation algorithms:<br>
    1)Region-Based IS.<br>
    2)Edge detection Based IS<br>
    3)Kmeans Clustering Based IS<br>
    4)Mask R-CNN model</p>

<h3>IS USING DCG-DGP </h3>

In [19]:
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import tensorflow as tf


In [3]:
from gpflow.likelihoods import MultiClass
from gpflow.kernels import RBF, White
from gpflow.models.svgp import SVGP
from gpflow.training import AdamOptimizer

from scipy.stats import mode
from scipy.cluster.vq import kmeans2

from doubly_stochastic_dgp.dgp import DGP

import time

In [20]:
#loading the data
def get_bsds_data(data_path='/data'):
    from tensorpack.dataflow.dataset import BSDS500
    train = BSDS500('train', data_dir=None, shuffle=True)
    test = BSDS500('test', data_dir=None, shuffle=True)
    val = BSDS500('val', data_dir=None, shuffle=True)

    X,Y  = train.data , train.label
    Xval,Yval = val.data , val.label
    Xtest,Ytest = test.data , test.label
    
    Y, Yval, Ytest = [np.array(y, dtype=float)[:, None] for y in [Y, Yval, Ytest]]

    X = np.concatenate([X, Xval], 0)
    Y = np.concatenate([Y, Yval], 0)
    
    return X.astype(float), Y.astype(float), Xtest.astype(float), Ytest.astype(float)

X, Y, Xs, Ys = get_bsds_data()


[0705 12:36:28 @fs.py:101] WRN Env var $TENSORPACK_DATASET not set, using C:\Users\DesignerBe\tensorpack_data for datasets.


In [11]:
import os
import numpy as np
import toml
import pandas
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

import seaborn
seaborn.set(style="white")

matplotlib.rcParams['axes.spines.top'] = False
matplotlib.rcParams['axes.spines.right'] = False
matplotlib.rcParams['axes.spines.bottom'] = False
matplotlib.rcParams['axes.spines.left'] = False
font = {'family' : 'Ubuntu, Arial',
        'size'   : 18}
matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=18)
matplotlib.rc('ytick', labelsize=18)
matplotlib.rc('figure', figsize=(11, 6.8))
matplotlib.rc('axes', titlesize=24)
matplotlib.rc('legend', fontsize=18)
matplotlib.rc('axes', labelsize=18)

In [14]:
RESULT_DIR = './results/'
def read_experiment(name):
    flags_path = os.path.join(RESULT_DIR, name, "options.toml")
    log_path = os.path.join(RESULT_DIR, name, "log.csv")
    try:
        log = pandas.read_csv(log_path)
    except pandas.errors.EmptyDataError:
        return None
    return {
        'log': log,
        'flags': toml.load(flags_path)
    }

In [16]:
experiments = os.listdir(RESULT_DIR)
experiments = [read_experiment(name) for name in experiments]
experiments = [e for e in experiments if e]

In [ ]:
plot_shape = (2, 2)
figure = plt.figure(figsize=(17, 10 * plot_shape[1]))

def plot_likelihood_accuracy(experiments, label_fn, row=0):
    axis = plt.subplot2grid(plot_shape, (row, 0)) 

    for i, experiment in enumerate(experiments):
        likelihood = experiment['log']['train_log_likelihood'].values
        global_step = experiment['log']['global_step'].values
        label = label_fn(experiment)
        axis.plot(global_step, likelihood, c="C{}".format(i), label=label)
        axis.set_xlabel('Optimization steps')
        axis.set_ylabel('Train likelihood')
    axis.legend();


    axis = plt.subplot2grid(plot_shape, (row, 1))

    for i, experiment in enumerate(experiments):
        test_accuracy = experiment['log']['test_accuracy'].values
        global_step = experiment['log']['global_step'].values
        label = label_fn(experiment)
        axis.plot(global_step, test_accuracy, c="C{}".format(i), label=label)
        axis.set_xlabel('Optimization steps')
        axis.set_ylabel('Test accuracy')

    axis.legend();
    plt.tight_layout();
    
    
M100_experiments = [e for e in experiments if e['flags']['M'] == 100]
M100_experiments.sort(key=lambda a: -a['flags']['N'])
plot_likelihood_accuracy(M100_experiments, lambda e: "N={}".format(e['flags']['N']))


M_experiments = [e for e in experiments if e['flags']['N'] == 60000 and e['flags']['fashion'] == False]
M_experiments.sort(key=lambda a: -a['flags']['M'])
plot_likelihood_accuracy(M_experiments, lambda e: "M={}".format(e['flags']['M']), row=1)

In [ ]:
def tile_inducing_points(points):
    points = points.reshape(-1, 5, 5)
    rows = int(np.ceil(np.sqrt(points.shape[0])))
    columns = int(np.ceil(np.sqrt(points.shape[0])))
    for row_index in range(rows):
        for column_index in range(columns):
            point_index = row_index * rows + column_index
            if point_index >= points.shape[0]:
                break;
            current_point = points[point_index]
            axis = plt.subplot2grid((rows, columns), (row_index, column_index))
            axis.imshow(current_point)
            axis.yaxis.set_visible(False)
            axis.xaxis.set_visible(False)

z_init = np.load('../results/N60000_M512/z_init.npy')

figure = plt.figure(figsize=(50, 50))
tile_inducing_points(z_init)
plt.tight_layout();

In [ ]:
inducing_points = np.load('../results/N60000_M512/inducing_points.npy')
figure = plt.figure(figsize=(50, 50))
tile_inducing_points(inducing_points)
plt.tight_layout()

In [3]:
import scipy.io
mat = scipy.io.loadmat('C:/Users/DesignerBe/ISDGP/BSR/BSDS500/data/groundTruth/train/2092.mat')

In [18]:
print(mat['groundTruth'])

[[array([[(array([[ 1,  1,  1, ...,  1,  1,  1],
       [ 1,  1,  1, ...,  1,  1,  1],
       [ 1,  1,  1, ...,  1,  1,  1],
       ...,
       [15, 15, 15, ..., 15, 15, 15],
       [15, 15, 15, ..., 15, 15, 15],
       [15, 15, 15, ..., 15, 15, 15]], dtype=uint16), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8))]],
      dtype=[('Segmentation', 'O'), ('Boundaries', 'O')])
  array([[(array([[ 1,  1,  1, ...,  1,  1,  1],
       [ 1,  1,  1, ...,  1,  1,  1],
       [ 1,  1,  1, ...,  1,  1,  1],
       ...,
       [10, 10, 10, ..., 10, 10, 10],
       [10, 10, 10, ..., 10, 10, 10],
       [10, 10, 10, ..., 10, 10, 10]], dtype=uint16), array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 

In [21]:
X.shape

(300, 321, 481, 3)

In [32]:
Ys.reshape(200,-1,1).shape

(200, 154401, 1)

In [27]:
np.concatenate([X[0:200],Xs],axis=0).shape

(400, 321, 481, 3)

In [34]:
X.shape[0]

300

In [37]:
X.mean(axis=(0, 1, 2))

array([ 93.83644519, 112.19120008, 109.93084829])